ZHAW, CAS Information Engineering, Module Big Data, December 2019  
  
Final Project **Christophe Otter & Severin Troesch**

## Movie Finder: Performance gain with Spark
  
##### *V7 - 06.01.2019*

### 1. Preparation Python

What we want in this section is the following:  
  
- Install and load the necessary modules
- Load the dataset (written plots of 35k movies)
- Convert the data into a python list for the "baseline" assessment of the performance

First, lets download and import the necessary modules:

In [4]:
## installing the external modules via library utilities (are available on notebook session level)
dbutils.library.installPyPI('nltk')
import nltk
nltk.download('stopwords')
nltk.download('punkt') #used for nltk.word_tokenize function

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Unzipping tokenizers/punkt.zip.
 Out[ 1 ]: True

In [5]:
## import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import re
# import nltk #language processing, imported above
from nltk.corpus import stopwords
stopw = set(stopwords.words('english')) # set of stopwords
import math
import json
import numpy

Then, lets get the data (upload movies-database as .csv manually to /FileStore/tables/):  
Path: /FileStore/tables/movies_df.csv

Now, we load the csv with pandas - and convert the relevant column into a python-list.

In [8]:
## Then, load csv, but with regular pandas-commands - for speed comparison later
movies_pd = pd.read_csv("/dbfs/FileStore/tables/movies_df.csv")

In [9]:
## have a look
movies_pd.head() #looks ok - a regular pandas df that is

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,All
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...","A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,The earliest known adaptation of the classic f...


In [10]:
# and make list of "All" variable to iterate over later (for the baseline comparison)
movies_ls = movies_pd.All.tolist()
type(movies_ls)

Out[ 11 ]: list

In [11]:
## check content
movies_ls[10] #good

Out[ 6 ]: 'The Rarebit Fiend gorges on Welsh rarebit at a restaurant. When he leaves, he begins to get dizzy as he starts to hallucinate. He desperately tries to hang onto a lamppost as the world spins all around him. A man helps him get home. He falls into bed and begins having more hallucinatory dreams. During a dream sequence, the furniture begins moving around the room. Imps emerge from a floating Welsh rarebit container and begin poking his head as he sleeps. His bed then begins dancing and spinning wildly around the room before flying out the window with the Fiend in it. The bed floats across the city as the Fiend floats up and off the bed. He hangs off the back and eventually gets caught on a weathervane atop a steeple. His bedclothes tear and he falls from the sky, crashing through his bedroom ceiling. The Fiend awakens from the dream after falling out of his bed. Dream of a Rarebit Fiend Dream of a Rarebit Fiend Dream of a Rarebit Fiend Dream of a Rarebit Fiend Dream of a Rarebit Fiend Dream of a Rarebit Fiend Dream of a Rarebit Fiend Dream of a Rarebit Fiend Dream of a Rarebit Fiend Dream of a Rarebit Fiend American American American American American American American American American American Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter Wallace McCutcheon and Edwin S. Porter nan nan nan nan nan nan nan nan nan nan shortshortshortshortshortshortshortshortshortshort'

### 2. Python helper Functions

What we want in this section is the following:

- Define the helper functions for the movie finder - them same functions that were used in the python-implementation.

Now, define the different helper functions necessary for the analysis of the dataset and for the movie-selection.  
  
Fow now, just take the same functions as in the spyder-pathon-version of the movie-recommender:

In [14]:
## normalizer function

def nor(txt):
    
    ''' doc:
       input (txt) = text string that needs to be normalised
       output = a list of normalised tokens - NOT UNIQUE!
    '''
    
    ## tokenize
    #txt_2 = nltk.word_tokenize(txt) # this causes problems in .map functionality later...
    txt_2 = txt.split() #tokenize 
    
    ## remove stopwords
    txt_2 = [word for word in txt_2 if word not in stopw]
    
    ## remove special characters and capitals
    txt_2 = [re.sub(r'\W+', '', word).lower() for word in txt_2]
    
    ## remove empty indexes
    txt_2 = [word for word in txt_2 if len(word)>1]
    
    # return result: a list of normalised tokens - NOT UNIQUE! - and joined with named entities
    return txt_2

In [15]:
## check
nor("bla is a word") #works

Out[ 6 ]: ['bla', 'word']

In [16]:
## function for TF out of list of not-unique normalised tokens (result from nor() function)
    
def tf(lis):
    
    ''' doc:
       input: 
           lis = list of not-unique normalised tokens (result from nor() function)
       output = a dict  with token and term freqencies
    '''
        
    res = {} #dict of tokens in lis (key) and # occurances in lis (value)
    cindex = set() #set of tokens in txt
    
    nt = len(lis) # number of tokens in txt
    
    # for every word in list
    for t in lis:
        if t not in cindex:
            cindex.add(t)
            res[t] = 1/nt # /nt gives the normalisation for the term-frequency
        else:
            res[t] += 1/nt
    
    ## return result - a dict of token and term freqencies
    return res


In [17]:
## idf function

# function that calculates idf for tokens in a collection of documents
# returns a dict (key = token, velue = idf)

def idf(coll):
    
    """doc:
    in: coll = list of of documents (strings of multiple tokens)
    out: dictionary of tokens to idf of token in collection
    """ 
    ts1 = round(time.time(),0) #timestamp 1
    
    N = len(coll)  
    print("Analysing {} documents...".format(N))
    print()
    
    dic = {}
    
    # get a list of sets of normalised tokens
    lss = [set(nor(doc)) for doc in coll] #returns a list of sets
    
    ts2 = round(time.time(),0) #timestamp 2
    print("step 1 (normalising & TF of all movie-plot strings) done ... (took {} s)".format(ts2-ts1))
    
    # set of all words in coll
    settot = set()
    for s in lss:
        settot = settot.union(s)
        
    ts3 = round(time.time(),0) #timestamp 3
    print("step 2 (merge all sets of normalised strings to one set) done ... (took {} s)".format(ts3-ts2))
        
    # for all tokens in collection 
    for tok in list(settot):
        nrdoc = 0
        
        # for all sets of tokens (i.e. documents)
        for tokset in lss:
            if tok in tokset:
                nrdoc += 1 #nrdoc: in how many docs is the token?
        
        dic[tok] = N/nrdoc #idf = N / number of docs that contain tok
    
    ts4 = round(time.time(),0) #timestamp 4 
    print("step 3 (for all tokens in all movies: determine idf) done ... (took {} s)".format(ts4-ts3))
    print()
    print("finished. (overall analysis time: {} s)".format(ts4 - ts1))
    return dic

In [18]:
## tfidf function: combine tf with idf - function takes string and gives dict of tokens:tfidf

def tfidf(txt, idfs):
    """ Compute TF-IDF
    Args:
        txt: string
        idfs (dictionary): token to IDF value
    Returns:
        dictionary: a dictionary of records to TF-IDF values
    """
    tfs = tf(nor(txt)) #a dict of token and term freqencies
    
    resdict = {}
    
    for t in tfs:
        if t in idfs:
            resdict[t] = tfs[t] * idfs[t] #calculate TF-IDF
        else:
            resdict[t] = 0 #zero if word is NOT in idfs reference
            
    return resdict

Now the ranking function (only cossim-function is applied, as the comparison between the ranking functions is not the point of this project):

In [20]:
## implement cosine-similarity function(s)

# dot product and normalizing funs
def dotprod(a, b):
    return sum([a[t] * b[t] for t in a if t in b])

def norm(a):
    return math.sqrt(dotprod(a, a))

# and finally, the actual fun to calculate cossim from two dicts
def cossim(dict1, dict2):
    """ Compute cosine similarity between two strings
    Args:
        dict1: first dict of tokens and tf-idf values
        dict2: second dict of tokens and tf-idf values

    Returns:
        cossim: cosine similarity value
    """

    return dotprod(dict1, dict2) / norm(dict1) / norm(dict2)

In [21]:
## build ranking-function to compare query and corpus  - using cosine similarity
    
def get_ranking_cossim(query, ref_dict, threshold, idf_dict):
    
    """ calculate and return a ranking of movies relative to searchstring (query)
    input:
        query = searchstring
        ref_dict = reference dictionary of movies (title:dict(token:tfidf))
        threshold = threshold reference score (default = 1) above which results are returned
        idfs = reference dict with term idfs
    output:
        a sorted listof tuples (title, relevance score) with movies in decreasing similarity scores-order
    """
    
    ## first, apply tf() and nor() function to query
    q = tfidf(query, idf_dict) #dict of query tokens:tfidf
    
    ## second, iterate over ref_dict and compare query to values (calculate relevance score)
        ## the result is a dict with movie titles as key and the relevance score (rs) as value
    res = {}
    
    for film in ref_dict: #all films in ref_dict
        
        
        # calculate relevance score (cosine similarity)
        rs = cossim(q,ref_dict[film])
        
        # add relevance score for film to res-dict
        res[film] = rs 
        
    ## define a sorted list with decreasing relevance scores
    result1 = sorted(res.items(), key=lambda kv: kv[1], reverse = True)
    # and choose items over threshold
    result2 = [mov[0] for mov in result1 if mov[1] > threshold]
             
    ## finally, return result
    return result2

###3. Apply and time code from python-implementation

What we want in this section is the following:

- Apply the above-defined helper functions - build the same movie recommender as in the python-implementation.
- Time the crucial steps - as baseline for later performance improvement (the two crucial parts are: idfs and res_dict)

In [23]:
## first, prepare the timing of the python-implementation
py_pre = time.time()

In [24]:
## define number of rows in df that should be used for development
n_test = 1000

In [25]:
## idf-dict: make dict of idfs for movies (for lookup)

# apply to n_test cases
#idfs = idf(movies_ls[:n_test])

# apply to whole list of movies
idfs = idf(movies_ls)

Analysing 34886 documents...

step 1 (normalising & TF of all movie-plot strings) done ... (took 33.0 s)
step 2 (merge all sets of normalised strings to one set) done ... (took 329.0 s)
step 3 (for all tokens in all movies: determine idf) done ... (took 788.0 s)

finished. (overall analysis time: 1150.0 s)

In [26]:
## check content
idfs['steamboat'] #worked

Out[ 17 ]: 1836.1052631578948

In [27]:
## apply functions to  dataset - a dict results (title:list of tuples(title,TF))

res_dict = {} #dictionary with titles (keys) and result dict (values). result dict is token:tfidf
ts_resdict1 = time.time() #timestamp 1

# loop over movies  
#for i, row in movies_pd.head(n_test).iterrows(): #only for n_test cases
for i, row in movies_pd.iterrows(): #for all cases

    txt = row["All"]
    n_mov = i+1
    
    res_dict[row["Title"]] = tfidf(txt, idfs) 

ts_resdict2 = time.time() #timestamp 2
secs = round(ts_resdict2 - ts_resdict1,1) #analysis time

print("Analysed {} movies in {} seconds.".format(n_mov, secs))

Analysed 34886 movies in 45.1 seconds.

In [28]:
## check content of result
res_dict['The Girl Who Stayed at Home'] # looks ok

Out[ 19 ]: {'atoline': 157.14414414414415,
 'additional': 0.6892287023865971,
 'seymour': 1.4550383717050384,
 'cabaret': 2.014668514668515,
 'dw': 78.57207207207207,
 'goes': 0.01681045615577066,
 'adoration': 7.142915642915643,
 'harron': 9.82150900900901,
 'the': 0.07323222251377592,
 'draft': 3.612509060784923,
 'who': 0.8205960529720322,
 'le': 1.8272574900481877,
 'rescue': 0.06732825370357504,
 'blossoms': 0.9189716031821294,
 'older': 0.11718429839235207,
 'return': 0.02563526005614097,
 'younger': 0.07748725056417363,
 'lestina': 576.1951951951952,
 'as': 0.01697937808148505,
 'confederate': 1.0476276276276275,
 'after': 0.012422461987679378,
 'carol': 4.401796754737932,
 'adolf': 19.163920017578555,
 'described': 0.46769090519090517,
 'hole': 0.3022002772002772,
 'film': 0.017906123990900656,
 'flag': 0.7447589769864651,
 'threatened': 0.2836536897908739,
 'monsieur': 8.270744428639166,
 'enlists': 0.37326400034238516,
 'when': 0.024655863206110323,
 'makes': 0.027112516242951024,
 'meet': 0.0300065197907474,
 'evade': 0.543751363820568,
 'lives': 0.06644572691084319,
 'father': 0.016347045058165418,
 'swears': 0.543751363820568,
 'behind': 0.038945264967569795,
 'france': 1.587314587314587,
 'griffith': 8.928644553644554,
 'faithful': 0.7896690660509755,
 'lines': 0.32135816798393485,
 'world': 0.03790259144817755,
 'soldier': 0.20172547386924794,
 'continue': 0.07562278351498755,
 'dempster': 192.06506506506508,
 'blossom': 5.1522670211194805,
 'german': 0.5693628411019716,
 'brothers': 0.17392821709368472,
 'officer': 0.05065897619089109,
 'allegiance': 2.014668514668515,
 'shell': 0.4895456203867419,
 'remain': 0.1533113601406284,
 'jim': 0.4742781010386644,
 'singer': 0.13172183079978553,
 'magazine3': 3.5714578214578214,
 'brother': 0.032076779780392764,
 'sweetheart': 0.385157216039569,
 'comes': 0.020687749360735145,
 'caught': 0.07165715647247795,
 'another': 0.021005767162698055,
 'sent': 0.050014049695781077,
 'dramadramadramadramadramadramadramadramadramadrama': 0.026348783391036912,
 'seeks': 0.13724379401235298,
 'him': 0.010422772709699817,
 'adventures': 0.45417382700619696,
 'trenches': 4.02933702933703,
 'beautiful': 0.0845770420582046,
 'cutie': 104.76276276276276,
 'civil': 0.3484349094105192,
 'son': 0.022027494273078797,
 'stayed': 8.97966537966538,
 'barthelmess': 7.483054483054483,
 'shot': 0.04836692648326997,
 'training': 0.16284367268823227,
 'grey': 0.85871117018658,
 'befriended': 0.6953280714342661,
 'patrol': 0.4081666081666082,
 'ralph': 0.6186777328509612,
 'catches': 0.10448413839371287,
 'james': 0.06285765765765766,
 'sweethearts': 2.7569148095463882,
 'girl': 0.2768571954618466,
 'american': 0.10503117576750194,
 'two': 0.013023714913322075,
 'promises': 0.09495114449797229,
 'war': 0.08918509883322596,
 'home': 0.1582837874135215,
 'man': 0.015506625630959557}

Recommender should now work:

In [30]:
## check it it works
get_ranking_cossim("France action drama new test movie tarantino", 
                   ref_dict = res_dict, 
                   threshold = 0.03, 
                   idf_dict = idfs)

# works.

Out[ 53 ]: ["My Best Friend's Birthday",
 'Reservoir Dogs',
 'From Dusk Till Dawn',
 'Kill Bill Volume 2',
 'Kill Bill Volume 1',
 'Girl 6',
 'Jackie Brown',
 'Django Unchained',
 'Pulp Fiction',
 'Inglourious Basterds']

In [31]:
## and manifest the timing for the python implementation
py_post = time.time()
print("The python-implementation took {} minutes overall.".format(round((py_post - py_pre)/60,2)))

The python-implementation took 19.98 minutes overall.

###4. Create the spark data structures

What we want in this section is the following:

- Create a spark-dataframe of movie-df for later parallelization
- Create an RDD of the "All" variable list for later parallelization

So lets create the spark dataframe:

In [34]:
## load data from csv file - create DataFrame (notabene: this is an RDD)
'''
movies = sqlContext.read.format("csv")\
.option("header","true")\
.option("delimiter", ",")\
.option("inferSchema", "true")\
.load("/FileStore/tables/movies_df.csv")
'''

# this does not work somehow - the resulting dataframe has weiredly shifted columns

Out[ 24 ]: '\nmovies = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").option("inferSchema", "true").load("/FileStore/tables/movies_df.csv")\n'

In [35]:
## better: create spark dataframe from pandas df
movies_spk = sqlContext.createDataFrame(movies_pd.astype(str))

In [36]:
## cache the df
movies_spk.cache()

Out[ 5 ]: DataFrame[Release Year: string, Title: string, Origin/Ethnicity: string, Director: string, Cast: string, Genre: string, Wiki Page: string, Plot: string, All: string]

In [37]:
## check form
movies_spk.show(36) # looks ok

+------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Release Year| Title|Origin/Ethnicity| Director| Cast| Genre| Wiki Page| Plot| All|
+------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 1901|Kansas Saloon Sma...| American| Unknown| nan| unknown|https://en.wikipe...|A bartender is wo...|A bartender is wo...|
 1901|Love by the Light...| American| Unknown| nan| unknown|https://en.wikipe...|The moon, painted...|The moon, painted...|
 1901|The Martyred Pres...| American| Unknown| nan| unknown|https://en.wikipe...|The film, just ov...|The film, just ov...|
 1901|Terrible Teddy, t...| American| Unknown| nan| unknown|https://en.wikipe...|Lasting just 61 s...|Lasting just 61 s...|
 1902|Jack and the Bean...| American|George S. Fleming...| nan| unknown|https://en.wikipe...|The earliest know...|The earliest know...|
 1903| Alice in Wonderland| American| Cecil Hepworth| May Clark| unknown|https://en.wikipe...|Alice follows a l...|Alice follows a l...|
 1903|The Great Train R...| American| Edwin S. Porter| nan| western|https://en.wikipe...|The film opens wi...|The film opens wi...|
 1904| The Suburbanite| American| Wallace McCutcheon| nan| comedy|https://en.wikipe...|The film is about...|The film is about...|
 1905|The Little Train ...| American|Edwin Stanton Porter| nan| unknown|https://en.wikipe...|The opening scene...|The opening scene...|
 1905|The Night Before ...| American|Edwin Stanton Porter| nan| unknown|https://en.wikipe...|Scenes are introd...|Scenes are introd...|
 1906|Dream of a Rarebi...| American|Wallace McCutcheo...| nan| short|https://en.wikipe...|The Rarebit Fiend...|The Rarebit Fiend...|
 1906|From Leadville to...| American|Francis J. Marion...| nan|short action/crim...|https://en.wikipe...|The film features...|The film features...|
 1906| Kathleen Mavourneen| American| Edwin S. Porter| nan| short film|https://en.wikipe...|Irish villager Ka...|Irish villager Ka...|
 1907| Daniel Boone| American|Wallace McCutcheo...|William Craven, F...| biographical|https://en.wikipe...|Boone's daughter ...|Boone's daughter ...|
 1907|How Brown Saw the...| American| Unknown| Unknown| comedy|https://en.wikipe...|Before heading ou...|Before heading ou...|
 1907| Laughing Gas| American|Edwin Stanton Porter|Bertha Regustus, ...| comedy|https://en.wikipe...|The plot is that ...|The plot is that ...|
 1908|The Adventures of...| American| D. W. Griffith|Arthur V. Johnson...| drama|https://en.wikipe...|On a beautiful su...|On a beautiful su...|
 1908| The Black Viper| American| D. W. Griffith| D. W. Griffith| drama|https://en.wikipe...|A thug accosts a ...|A thug accosts a ...|
 1908|A Calamitous Elop...| American| D.W. Griffith|Harry Solter, Lin...| comedy|https://en.wikipe...|A young couple de...|A young couple de...|
 1908|The Call of the Wild| American| D. W. Griffith| Charles Inslee| adventure|https://en.wikipe...|A white girl (Flo...|A white girl (Flo...|
 1908| A Christmas Carol| American| Unknown| Tom Ricketts| drama|https://en.wikipe...|No prints of the ...|No prints of the ...|
 1908|The Fight for Fre...| American| D. W. Griffith|Florence Auer, Jo...| western|https://en.wikipe...|The film opens in...|The film opens in...|
 1909| At the Altar| American| D. W. Griffith| Marion Leonard| drama|https://en.wikipe...|A rejected admire...|A rejected admire...|
 1909|A Drunkard's Refo...| American| D. W. Griffith| Arthur V. Johnson| drama|https://en.wikipe...|John Wharton, the...|John Wharton, the...|
 1909| The Golden Louis| American| D. W. Griffith| nan| drama|https://en.wikipe...|An old woman send...|An old woman send...|
 1909|The Lure of the Gown| American| D.W. Griffith| Marion Leonard| unknown|https://en.wikipe...|The story as told...|The story as told...|
 1910| An Arcadian Maid| Amer

In [38]:
## check type
movies_spk #its a spark dataframe - good

Out[ 15 ]: DataFrame[Release Year: string, Title: string, Origin/Ethnicity: string, Director: string, Cast: string, Genre: string, Wiki Page: string, Plot: string, All: string]

In [39]:
## have a look at schema
movies_spk.printSchema() #looks good

root
-- Release Year: string (nullable = true)
-- Title: string (nullable = true)
-- Origin/Ethnicity: string (nullable = true)
-- Director: string (nullable = true)
-- Cast: string (nullable = true)
-- Genre: string (nullable = true)
-- Wiki Page: string (nullable = true)
-- Plot: string (nullable = true)
-- All: string (nullable = true)

In [40]:
## To test: Perform first SQL-query on dataframe (result is still an RDD)
movies_spk.registerTempTable("mov")
res1 = sqlContext.sql("select * from mov\
                      where Title == 'Inception'")
res1.show()

# (takes about 4 s)

+------------+---------+----------------+-----------------+--------------------+---------------+--------------------+--------------------+--------------------+
Release Year| Title|Origin/Ethnicity| Director| Cast| Genre| Wiki Page| Plot| All|
+------------+---------+----------------+-----------------+--------------------+---------------+--------------------+--------------------+--------------------+
 2010|Inception| American|Christopher Nolan|Leonardo DiCaprio...|science fiction|https://en.wikipe...|Dominick "Dom" Co...|Dominick "Dom" Co...|
+------------+---------+----------------+-----------------+--------------------+---------------+--------------------+--------------------+--------------------+

In [41]:
## further test: run some spark transformations and actions:
movies_spk.select("Title", "Release Year").filter(movies_spk["Release Year"] > 2016).show(10) #works

+--------------------+------------+
 Title|Release Year|
+--------------------+------------+
Underworld: Blood...| 2017|
 Monster Trucks| 2017|
 The Bye Bye Man| 2017|
 Sleepless| 2017|
 100 Streets| 2017|
 The Book of Love| 2017|
 Split| 2017|
xXx: Return of Xa...| 2017|
The Resurrection ...| 2017|
 Trespass Against Us| 2017|
+--------------------+------------+
only showing top 10 rows

Try to save the dataframe as parquet file... DOESNT WORK SO FAR

In [43]:
## write a parquet file for later use - DOESNT WORK...

# Remove the file if it exists
#dbutils.fs.rm("/tmp/movies_pq", True)

# then write
#movies_spk.write.parquet("/tmp/movies_pq")
#movies_spk.write.parquet('C:\\Users\\sever\\Google Drive\\Dokumente PC\\ZHAW\\CAS Information Engineering\\Modul Big Data\\project\\performance_analysis_movie_recommender\\movies_pq')

Now, save the python list with the "All" variable as RDD:

In [45]:
## python list to RDD
par_deg = "automatically chosen" #degree of parallelization - or is it stupid to set that manually?
movies_ls_rdd = sc.parallelize(movies_ls) #degree of parallelization chosen automatically

In [46]:
## check the dada type
movies_ls_rdd # an RDD - good

Out[ 18 ]: ParallelCollectionRDD[10] at readRDDFromFile at PythonRDD.scala:330

In [47]:
## run some basic operations on RDD
print(movies_ls_rdd.count())
print(movies_ls_rdd.takeSample(False, 1))
print(movies_ls_rdd.take(1))
#works

34886
['Frankie is a hard-drinking woman fresh out of a bad marriage. She wanders East Los Angeles looking for castoff furniture and clothing, which she sometimes sells for spending money. Otherwise, she hangs out with local musicians as an opportunity to meet people.\r\r\nLev is a limousine driver who dreams of making it in the music business. He meets Frankie in a dive bar and they hook up, quickly escalating to the point where she moves in with him. Lev also makes a musical connection with Charlie King Nash, a well-known roots-rocker who has hit a creative wall and welcomes the chance to make a new start.\r\r\nMeantime, Lev and Frankie try to work through the ups and downs of a serious relationship and decide whether each is ready for it.[1][2] 9 Full Moons 9 Full Moons 9 Full Moons 9 Full Moons 9 Full Moons 9 Full Moons 9 Full Moons 9 Full Moons 9 Full Moons 9 Full Moons American American American American American American American American American American Tomer Almagor Tomer Almagor Tomer Almagor Tomer Almagor Tomer Almagor Tomer Almagor Tomer Almagor Tomer Almagor Tomer Almagor Tomer Almagor Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue Amy Seimetz, Bret Roberts, Donal Logue dramadramadramadramadramadramadramadramadramadrama']
["A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1] Kansas Saloon Smashers Kansas Saloon Smashers Kansas Saloon Smashers Kansas Saloon Smashers Kansas Saloon Smashers Kansas Saloon Smashers Kansas Saloon Smashers Kansas Saloon Smashers Kansas Saloon Smashers Kansas Saloon Smashers American American American American American American American American American American Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown Unknown nan nan nan nan nan nan nan nan nan nan unknownunknownunknownunknownunknownunknownunknownunknownunknownunknown"]

In [48]:
## now a .map operation - which will be important later

# goal: only take first 10 characters of text and swho the first three movies
print(movies_ls_rdd.map(lambda text: text[0:10]).take(3)) #works

# goal: apply nor() function, take frst token of first three movies
print(movies_ls_rdd.map(lambda text: nor(text)).map(lambda ls: ls[0]).take(3)) #works

['A bartende', 'The moon, ', 'The film, ']
['bartender', 'the', 'the']

###5. Create the spark helper functions

What we want in this section is the following:

- Parallalize the analysis in the two crucial steps (i.e. formulate the python helper functions as parallalizable spark-functions)

So now, lets re-formulate the the python helper functions for the recommender in "spark language". So that i can be parallelized on multiple clusters:

In [51]:
## normalizer function
## THIS STAYS THE SAME AS IN THE PYTHON IMPLEMENTATION. NO CHANGE NEEDED.


def nor(txt):
    
    ''' doc:
       input (txt) = text string that needs to be normalised
       output = a list of normalised tokens - NOT UNIQUE!
    '''
    
    ## tokenize
    #txt_2 = nltk.word_tokenize(txt) # this causes problems in .map functionality later...
    txt_2 = txt.split() #tokenize 
    
    ## remove stopwords
    txt_2 = [word for word in txt_2 if word not in stopw]
    
    ## remove special characters and capitals
    txt_2 = [re.sub(r'\W+', '', word).lower() for word in txt_2]
    
    ## remove empty indexes
    txt_2 = [word for word in txt_2 if len(word)>1]
    
    # return result: a list of normalised tokens - NOT UNIQUE! - and joined with named entities
    return txt_2

In [52]:
## function for TF out of list of not-unique normalised tokens (result from nor() function)
## THIS STAYS THE SAME AS IN THE PYTHON IMPLEMENTATION. NO CHANGE NEEDED.
    
def tf(lis):
    
    ''' doc:
       input: 
           lis = list of not-unique normalised tokens (result from nor() function)
       output = a dict  with token and term freqencies
    '''
        
    res = {} #dict of tokens in lis (key) and # occurances in lis (value)
    cindex = set() #set of tokens in txt
    
    nt = len(lis) # number of tokens in txt
    
    # for every word in list
    for t in lis:
        if t not in cindex:
            cindex.add(t)
            res[t] = 1/nt # /nt gives the normalisation for the term-frequency
        else:
            res[t] += 1/nt
    
    ## return result - a dict of token and term freqencies
    return res


In [53]:
## idf function - SPARK VERSION

# function that calculates idf for tokens in a collection of documents
# returns a dict (key = token, velue = idf)

def idf_spk(coll):
    
    """doc:
    in: coll = list (RDD) of of documents (dociments = strings of multiple tokens)
    out: pair of tokens and idf of token in collection
    """ 
    
    # count dumber of documents in collection
    N = float(coll.count())
    
    # get a list of (non-unique) normalised tokens
    lss = coll.flatMap(lambda doc: list(set(nor(doc)))) 
    
    # build tuples: (token, 1) for all tokens in lss
    token_count = lss.map(lambda token: (token, 1))
    
    # reduce: build tuple with (token, sum of occurances)
    token_sum = token_count.reduceByKey(lambda a, b : a + b)
    
    # calculate idf and return pair with (token, idf)
    return (token_sum.map(lambda tok: (tok[0], float(N/tok[1]))))

In [54]:
## test function
idf_spk(sc.parallelize(movies_ls[:3],2)).collect() # seems ok
#sc.parallelize(movies_ls[:10],2).flatMap(lambda doc: list(set(nor(doc)))).map(lambda token: (token, 1)).reduceByKey(lambda a, b : a + b).collect()

Out[ 16 ]: [('eight', 3.0),
 ('saloon', 3.0),
 ('bartender', 3.0),
 ('fans', 3.0),
 ('working', 3.0),
 ('cash', 3.0),
 ('policemen', 3.0),
 ('last', 3.0),
 ('mirrors', 3.0),
 ('past', 3.0),
 ('after', 3.0),
 ('dumping', 3.0),
 ('viewing', 3.0),
 ('hat', 1.5),
 ('presidents', 3.0),
 ('garfield', 3.0),
 ('assassin', 3.0),
 ('altar', 3.0),
 ('burst', 3.0),
 ('bar', 3.0),
 ('mckinleyeach', 3.0),
 ('unknownunknownunknownunknownunknownunknownunknownunknownunknownunknown',
 1.0),
 ('group', 3.0),
 ('tree', 3.0),
 ('shoulder', 3.0),
 ('eyes', 3.0),
 ('long', 3.0),
 ('smiling', 3.0),
 ('fence', 3.0),
 ('nation', 3.0),
 ('portraits', 3.0),
 ('water', 3.0),
 ('stereotypically', 3.0),
 ('nations', 3.0),
 ('look', 3.0),
 ('center', 3.0),
 ('railing', 3.0),
 ('kneels', 3.0),
 ('head', 3.0),
 ('three', 3.0),
 ('bucket', 3.0),
 ('at', 3.0),
 ('bench', 3.0),
 ('irish', 3.0),
 ('sits', 3.0),
 ('in', 1.5),
 ('frown', 3.0),
 ('love', 3.0),
 ('young', 3.0),
 ('assassination', 3.0),
 ('scene', 3.0),
 ('tomb', 3.0),
 ('james', 3.0),
 ('smiles', 3.0),
 ('lady', 3.0),
 ('hidden', 3.0),
 ('night', 3.0),
 ('unknown', 1.0),
 ('moons', 3.0),
 ('sky', 3.0),
 ('appear', 3.0),
 ('fills', 3.0),
 ('fixtures', 3.0),
 ('two', 3.0),
 ('carrie', 3.0),
 ('everything', 3.0),
 ('sprays', 3.0),
 ('us', 3.0),
 ('painted', 3.0),
 ('up', 3.0),
 ('kansas', 3.0),
 ('face', 1.0),
 ('light', 3.0),
 ('second', 3.0),
 ('base', 3.0),
 ('walking', 3.0),
 ('martyred', 3.0),
 ('followers', 3.0),
 ('better', 3.0),
 ('victims', 3.0),
 ('the', 1.0),
 ('presidentsabraham', 3.0),
 ('displays', 3.0),
 ('feet', 3.0),
 ('mans', 3.0),
 ('seltzer', 3.0),
 ('customers', 3.0),
 ('sit', 3.0),
 ('nan', 1.0),
 ('lincoln', 3.0),
 ('seconds', 3.0),
 ('couple', 3.0),
 ('view', 3.0),
 ('justice', 3.0),
 ('causing', 3.0),
 ('gets', 3.0),
 ('first', 3.0),
 ('left', 3.0),
 ('embrace', 3.0),
 ('hangs', 3.0),
 ('serving', 3.0),
 ('portal', 3.0),
 ('smile', 3.0),
 ('perched', 3.0),
 ('minute', 3.0),
 ('assault', 3.0),
 ('girl', 3.0),
 ('woman', 3.0),
 ('park', 3.0),
 ('shots', 3.0),
 ('breaking', 3.0),
 ('william', 3.0),
 ('beer', 3.0),
 ('pulling', 3.0),
 ('drinks', 3.0),
 ('man', 1.5),
 ('shot', 3.0),
 ('see', 3.0),
 ('they', 1.5),
 ('runs', 3.0),
 ('film', 3.0),
 ('inside', 3.0),
 ('moon', 3.0),
 ('everybody', 3.0),
 ('smashers', 3.0),
 ('american', 1.0),
 ('begin', 3.0),
 ('smashing', 3.0),
 ('learn', 3.0),
 ('blocked', 3.0),
 ('composed', 3.0),
 ('leave1', 3.0),
 ('wrecking', 3.0),
 ('register', 3.0),
 ('camera', 3.0),
 ('bigger', 3.0),
 ('order', 3.0)]

In [55]:
# compare results of python and spark function!

res_py = idf(movies_ls[:3]) # this is the python result
print(res_py)

res_spk = idf_spk(sc.parallelize(movies_ls[:3],2))
print(res_spk.collect())

# seems ok!

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1388333490580178> in <module> () 
 1 # compare results of python and spark function! 
 2 
 ----> 3 res_py = idf ( movies_ls [ : 3 ] ) # this is the python result 
 4 print ( res_py ) 
 5 

 NameError : name 'idf' is not defined

In [56]:
## tfidf function: combine tf with idf - function takes string and gives dict of tokens:tfidf
## THIS STAYS THE SAME AS IN THE PYTHON IMPLEMENTATION. NO CHANGE NEEDED.

def tfidf(txt, idfs):
    """ Compute TF-IDF
    Args:
        txt: string
        idfs (dictionary): token to IDF value
    Returns:
        dictionary: a dictionary of records to TF-IDF values
    """
    tfs = tf(nor(txt)) #a dict of token and term freqencies
    
    resdict = {}
    
    for t in tfs:
        if t in idfs:
            resdict[t] = tfs[t] * idfs[t] #calculate TF-IDF
        else:
            resdict[t] = 0 #zero if word is NOT in idfs reference
            
    return resdict

Now the ranking function (only cossim-function is applied, as the comparison between the ranking functions is not the point of this project):

In [58]:
## implement cosine-similarity function(s)
## THESE STAY THE SAME AS IN THE PYTHON IMPLEMENTATION. NO CHANGE NEEDED.

# dot product and normalizing funs
def dotprod(a, b):
    return sum([a[t] * b[t] for t in a if t in b])

def norm(a):
    return math.sqrt(dotprod(a, a))

# and finally, the actual fun to calculate cossim from two dicts
def cossim(dict1, dict2):
    """ Compute cosine similarity between two strings
    Args:
        dict1: first dict of tokens and tf-idf values
        dict2: second dict of tokens and tf-idf values

    Returns:
        cossim: cosine similarity value
    """

    return dotprod(dict1, dict2) / norm(dict1) / norm(dict2)

In [59]:
## build ranking-function to compare query and corpus  - using cosine similarity
    
def get_ranking_cossim_spk(query, ref_pairs, threshold, idf_dict):
    
    """ calculate and return a ranking of movies relative to searchstring (query)
    input:
        query = searchstring
        ref_pairs = reference rdd pairs of movies (title:dict(token:tfidf))
        threshold = threshold reference score above which results are returned
        idf_dict = reference dict with term idfs as pairs of (token, idf)
    output:
        a sorted listof tuples (title, relevance score) with movies in decreasing similarity scores-order
    """
    
    ## first, apply tfidf() function to query
    q = tfidf(query, idf_dict) #yields dict of query tokens:tfidf 
    
    ## second, go over ref_pairs and compare query to values (calculate relevance score)
        ## the result is a pair with movie titles as key and the relevance score (rs) as value
      
    result = (ref_pairs 
              .map(lambda film: (film[0], cossim(q,film[1]))))
              
    ## and return unordered result
    return result #.takeOrdered(10, lambda s: -1*s[1]) #take the first 10 #---------------------------------- this is the problem... the sorting takes forever...


###6. Apply the spark helper functions

What we want in this section is the following:

- Parallalize the analysis in the two crucial steps (i.e. apply the helper functions)
- Time the (parallalized) crucial steps
- Plot speed of parallalized version compared to baseline (maybe in next section...)

In [61]:
## prepare the timing of the spark-implementation
spk_pre = time.time()

First, the idfs-reference dict

In [63]:
##First, apply the idf_spk function and build the spark idf-reference
idfs_spk = idf_spk(movies_ls_rdd)#.cache() #lazy evaluation - only executed when action is called

In [64]:
## check type
idfs_spk #RDD, good

Out[ 37 ]: PythonRDD[11] at RDD at PythonRDD.scala:57

In [65]:
## and show the results 
# idfs_spk.takeSample(False,3) #this is an action

#takes only 2 min (on databricks community version)?! even without multiple clusters??

In [66]:
# check result - seems ok
# print(idfs_spk.count()) # number of tokens with idf-result

In [67]:
## collect idfs dictionary as python dict to driver, and BROADCAST the variable
## idf_spk() function is evaluated here - therefore it takes a while

idfs_spk_dict = idfs_spk.collectAsMap() #we use the collectAsMap() action to return the IDFs to the driver as a Python dictionary.
idfs_spk_dict_bc = sc.broadcast(idfs_spk_dict) #and we save it as broadcsted variable

In [68]:
## check types
print(type(idfs_spk_dict))
print(type(idfs_spk_dict_bc))

<class 'dict'>
<class 'pyspark.broadcast.Broadcast'>

Now, make the "movie reference dict"

In [70]:
## apply tfidf to  dataset (lazy eval)
ts_resdict_spk1 = time.time() #timestamp 1

# go over movies (lazy evaluation)
movies_res_spk = (movies_spk #the dataframe of the whole dataset
                  .select("Title","All") #select the two important columns
                  .rdd #build an rdd (map functions cannot be applied to dataframes, it seems...)
                  .map(lambda row: (row[0],tfidf(row[1], idfs_spk_dict_bc.value))) #map to make pair of (title, tfidf-dict of text) ------------- SROCKINGER: here the broadcasted, correct?(not if the movies_res variable is broadcsted a well?!)
                  .cache()) 

ts_resdict_spk2 = time.time() #timestamp 2
secs_spk = round(ts_resdict_spk2 - ts_resdict_spk1,1) #analysis time

print("Analysed all movies in {} seconds. No surprise with lazy evaluation ;-)".format(secs_spk))

Analysed all movies in 0.2 seconds. No surprise with lazy evaluation ;-)

In [71]:
## check result - takes a while
print(movies_res_spk) #an RDD, good
#print(movies_res_spk.take(1)) #looks good
#print(movies_res_spk.count()) #seems correct

PythonRDD[15] at RDD at PythonRDD.scala:57

In [72]:
## collect and broadcast the movies_res_spk RDD (doesn't work if i try to broadcast the RDD directly (Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver,).)

#movies_res_spk_coll = movies_res_spk.collect() #collect to driver - takes a while - use this later to time the effect of the broadcast var

In [73]:
#  movies_res_spk_coll_bc = sc.broadcast(movies_res_spk_coll) # and broadcast
#  movies_res_spk_coll_bc #show type

# this works. however, i think its not necessary as the get_ranking_cossim_spk function takes an rdd as input

*Note:* The whole thing is way faster than with the python code - even without parallelization

##### Now test the whole recommender:

In [76]:
#quickly re-test tfidf() function
tfidf("a test bond", idfs_spk_dict_bc.value) # good

Out[ 46 ]: {'bond': 22.741851368970014, 'test': 18.24581589958159}

In [77]:
## then apply ranking-function (lazy evaluation again)
query_test = "France action drama new test movie tarantino"

result_q1 = get_ranking_cossim_spk(query = query_test, ref_pairs = movies_res_spk, threshold = 0.1, idf_dict = idfs_spk_dict_bc.value)
# result_q1 = get_ranking_cossim_spk(query = query_test, ref_pairs = movies_res_spk_coll_bc.value, threshold = 0.1, idf_dict = idfs_spk_dict_bc.value) #------- does not work with the broadcasted movies_res_spk_coll_bc

In [78]:
## type?
result_q1 #rdd

Out[ 27 ]: PythonRDD[27] at RDD at PythonRDD.scala:57

In [79]:
## and show results (get_ranking_cossim_spk function is evaluated here (and the definition of movies_res_spk is also only evaluated here) - therefore this step takes long with cold cache (i.e. not-yet-cached movies_res_spk))

#result_q1.take(100) #takes only 3s

# do the ordering on the result of the get_ranking_cossim_spk function:
result_q1.takeOrdered(10, lambda s: -1*s[1]) # this (sorting) used to take long! why??

# works!

## note:
# cold cache (first time): ca. 1.3 min
# warm cache (starting second time): under 2 seconds

# I think the point here might be the cashing of the movies_res_spk dict. Once this happened (and it happens in the first iteration of the code) the sorting is way faster...
# when the cashing does not happen (in the definition of the movies_res_spk variable) the code takes 1.2 min in any iteration!

Out[ 28 ]: [("My Best Friend's Birthday", 0.9545999944036624),
 ('Reservoir Dogs', 0.7145623063126616),
 ('From Dusk Till Dawn', 0.46822197465547655),
 ('Kill Bill Volume 2', 0.32420709837713296),
 ('Kill Bill Volume 1', 0.3174109851105095),
 ('Girl 6', 0.21924820278988907),
 ('Jackie Brown', 0.09054316914467836),
 ('Django Unchained', 0.0834438438437147),
 ('Pulp Fiction', 0.08048703631350722),
 ('Inglourious Basterds', 0.03678268856492387)]

In [80]:
## and manifest the timing for the python implementation
#spk_post = time.time()
print("The spark-implementation (20 cores) took {} seconds overall.".format(round((spk_post - spk_pre),1)))

The spark-implementation (20 cores) took 23.5 seconds overall.

###7. Multiply the dataset - for sepeed scaling experiment

What we want in this section is the following:

- multiply the two important RDDs (movies_spk, movies_ls_rdd). That is, create a new RDD that is just x times the old RDDs repeated.
- run the critical steps (i.e. ...) on 1, 2 and 4 nodes and see how the speed of the multiplied RDDs scale.

*Note:* "par_deg" in Cmd 43 has to be changed for the sensible use of multiple clusters... Best option is to choose the degree automatically.

In [82]:
## cjeck degree of parallelization
par_deg

Out[ 26 ]: 'automatically chosen'

In [83]:
## multiply the spark-datasets x 2

# multiply how many times?
multipl_1 = 2

# apply for movies_ls_rdd
movies_ls_rdd_mult2 = movies_ls_rdd.flatMap(lambda x: [x]*multipl_1)

# and for movies_spk
movies_spk_mult2 = movies_spk
itera1 = 1
while itera1 < multipl_1:
  movies_spk_mult2 = movies_spk_mult2.union(movies_spk)
  itera1 += 1

In [84]:
## multiply the spark-datasets x 4

# multiply how many times?
multipl_2 = 4

# apply for movies_ls_rdd
movies_ls_rdd_mult4 = movies_ls_rdd.flatMap(lambda x: [x]*multipl_2)

# and for movies_spk
movies_spk_mult4 = movies_spk
itera2 = 1
while itera2 < multipl_2:
  movies_spk_mult4 = movies_spk_mult4.union(movies_spk)
  itera2 += 1

In [85]:
## multiply the spark-datasets x 10

# multiply how many times?
multipl_3 = 10

# apply for movies_ls_rdd
movies_ls_rdd_mult10 = movies_ls_rdd.flatMap(lambda x: [x]*multipl_3)

# and for movies_spk
movies_spk_mult10 = movies_spk
itera3 = 1
while itera3 < multipl_3:
  movies_spk_mult10 = movies_spk_mult10.union(movies_spk)
  itera3 += 1

In [86]:
## see if it worked (numbers should be the same)
print(movies_ls_rdd.count() * 4)
print(movies_ls_rdd_mult2.count() * 2)
print(movies_ls_rdd_mult4.count())
print()
print(movies_spk.count() * 4)
print(movies_spk_mult2.count() * 2)
print(movies_spk_mult4.count())
print()
print(movies_spk_mult10.count()/10*4)

# looks ok

139544
139544
139544

139544
139544
139544

139544.0

Now apply the "critical function" (i.e. the calculation of the idfs ref dict) for the multiplied datasets:

In [88]:
##First, apply the idf_spk function and build the spark idf-reference
idfs_spk_mult1 = idf_spk(movies_ls_rdd) #lazy. this has been done above - but now new name for speed comparison
idfs_spk_mult2 = idf_spk(movies_ls_rdd_mult2) #a lazy evaluation
idfs_spk_mult4 = idf_spk(movies_ls_rdd_mult4) #again a lazy evaluation
idfs_spk_mult10 = idf_spk(movies_ls_rdd_mult10) #again a lazy evaluation

Now evaluate and compare running time - always make a list of 10 iterations of the *cold cache*-version of the respective analysis (action: count of total tokens - should be the same in all three cases as there are the same tokens)

In [90]:
## dataset x 1

# result list
times_dx1 = []

# timing, 10 times
for i in range(10):
  
  # re-define idfs - so that the count() action later is always "cold cache" and the whole calculation is actually executed
  idfs_spk_mult1 = idf_spk(movies_ls_rdd) #lazy.
  
  # then the timing
  t1_1 = time.time()
  idfs_spk_mult1.count()
  t2_1 = time.time()

  #append result to list
  times_dx1.append(t2_1 - t1_1)
  
  # show progress
  print("Iteration Nr. {} done ...".format(i+1))

print()
print("The mean for the data x1 is {} seconds.".format(round(numpy.mean(times_dx1),1)))

Iteration Nr. 1 done ...
Iteration Nr. 2 done ...
Iteration Nr. 3 done ...
Iteration Nr. 4 done ...
Iteration Nr. 5 done ...
Iteration Nr. 6 done ...
Iteration Nr. 7 done ...
Iteration Nr. 8 done ...
Iteration Nr. 9 done ...
Iteration Nr. 10 done ...

The mean for the data x1 is 6.9 seconds.

In [91]:
## dataset x 2

# result list
times_dx2 = []

# timing, 10 times
for i in range(10):
  
  # re-define idfs - so that the count() action later is always "cold cache" and the whole calculation is actually executed
  idfs_spk_mult2 = idf_spk(movies_ls_rdd_mult2)
  
  # then the timing
  t1_2 = time.time()
  idfs_spk_mult2.count()
  t2_2 = time.time()

  #append result to list
  times_dx2.append(t2_2 - t1_2)
  
  # show progress
  print("Iteration Nr. {} done ...".format(i+1))

print()
print("The mean for the data x2 is {} seconds.".format(round(numpy.mean(times_dx2),1)))

Iteration Nr. 1 done ...
Iteration Nr. 2 done ...
Iteration Nr. 3 done ...
Iteration Nr. 4 done ...
Iteration Nr. 5 done ...
Iteration Nr. 6 done ...
Iteration Nr. 7 done ...
Iteration Nr. 8 done ...
Iteration Nr. 9 done ...
Iteration Nr. 10 done ...

The mean for the data x2 is 12.2 seconds.

In [92]:
## dataset x 4

# result list
times_dx4 = []

# timing, 10 times
for i in range(10):
  
  # re-define idfs - so that the count() action later is always "cold cache" and the whole calculation is actually executed
  idfs_spk_mult4 = idf_spk(movies_ls_rdd_mult4)
  
  # then the timing
  t1_4 = time.time()
  idfs_spk_mult4.count()
  t2_4 = time.time()

  #append result to list
  times_dx4.append(t2_4 - t1_4)
  
  # show progress
  print("Iteration Nr. {} done ...".format(i+1))

print()
print("The mean for the data x4 is {} seconds.".format(round(numpy.mean(times_dx4),1)))

Iteration Nr. 1 done ...
Iteration Nr. 2 done ...
Iteration Nr. 3 done ...
Iteration Nr. 4 done ...
Iteration Nr. 5 done ...
Iteration Nr. 6 done ...
Iteration Nr. 7 done ...
Iteration Nr. 8 done ...
Iteration Nr. 9 done ...
Iteration Nr. 10 done ...

The mean for the data x4 is 24.1 seconds.

In [93]:
## dataset x 10

# result list
times_dx10 = []

# timing, 10 times
for i in range(10):
  
  # re-define idfs - so that the count() action later is always "cold cache" and the whole calculation is actually executed
  idfs_spk_mult10 = idf_spk(movies_ls_rdd_mult10)
  
  # then the timing
  t1_10 = time.time()
  idfs_spk_mult10.count()
  t2_10 = time.time()

  #append result to list
  times_dx10.append(t2_10 - t1_10)
  
  # show progress
  print("Iteration Nr. {} done ...".format(i+1))

print()
print("The mean for the data x10 is {} seconds.".format(round(numpy.mean(times_dx10),1)))

Iteration Nr. 1 done ...
Iteration Nr. 2 done ...
Iteration Nr. 3 done ...
Iteration Nr. 4 done ...
Iteration Nr. 5 done ...
Iteration Nr. 6 done ...
Iteration Nr. 7 done ...
Iteration Nr. 8 done ...
Iteration Nr. 9 done ...
Iteration Nr. 10 done ...

The mean for the data x10 is 58.0 seconds.

Keep this for the DAG-visualization

In [95]:
# dataset x 10
idfs_spk_mult10.count()

Out[ 34 ]: 199210

Now, time the effect of the broadcast-variable in the

In [97]:
## apply tfidf to 4x dataset (lazy eval) - same as above, just again to time the broadcast-effect

# go over movies (lazy evaluation) - WITH BROADCAST
movies_res_spk_bc = (movies_spk_mult10 #the dataframe of the whole dataset
                  .select("Title","All") #select the two important columns
                  .rdd #build an rdd (map functions cannot be applied to dataframes, it seems...)
                  .map(lambda row: (row[0],tfidf(row[1], idfs_spk_dict_bc.value))) #here be the broadcast
                  .cache()) 

# go over movies (lazy evaluation) - NO BROADCAST
movies_res_spk_nbc = (movies_spk_mult10 #the dataframe of the whole dataset
                  .select("Title","All") #select the two important columns
                  .rdd #build an rdd (map functions cannot be applied to dataframes, it seems...)
                  .map(lambda row: (row[0],tfidf(row[1], idfs_spk_dict))) #here be the broadcast
                  .cache()) 

In [98]:
## broadcasted
movies_res_spk_coll_bc = movies_res_spk_bc.count() #collect to driver

In [99]:
movies_res_spk_coll_bc

Out[ 45 ]: 348860

In [100]:
## non-broadcasted
movies_res_spk_coll_nbc = movies_res_spk_nbc.count() #collect to driver

In [101]:
movies_res_spk_coll_nbc

Out[ 47 ]: 348860

The broadcast does not seem to do much... For reasons see the following text (from big data exercise week 4):

"The solution in (3c) works well for small datasets, but it requires Spark to (automatically) send the idfsSmallWeights variable to all the workers. If we didn't cache() similarities, then it might have to be recreated if we run similar() multiple times. This would cause Spark to send idfsSmallWeights every time.
Instead, we can use a broadcast variable - we define the broadcast variable in the driver and then we can refer to it in each worker. Spark saves the broadcast variable at each worker, so it is only sent once."

Mayve, in our case, the idfs-dict is sent only once to each worker anyway... therefore its no wonder that the broadcast does not do much...?